# Aktywne uczenie
## Przygotowanie środowiska

In [2]:
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
    print(device)

cuda


In [4]:
# GPU operations have a separate seed we also want to set
if torch.cuda.is_available(): 
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    
# Additionally, some operations on a GPU are implemented stochastic for efficiency
# We want to ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = True #as True useful with training CNN networks

## Przygotowanie danych - CIFAR10

In [5]:
transform = transforms.Compose(
    [transforms.Resize((64, 64)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Generate starting indices

In [8]:
def get_train_val():
    rand_sampler = torch.utils.data.RandomSampler(trainset, replacement=False)
    rand_batch_sampler = torch.utils.data.BatchSampler(rand_sampler,
                                                       batch_size=int(0.1*len(trainset)),
                                                       drop_last=False)
    train_idx = []
    for i, batch in enumerate(iter(rand_batch_sampler)):
        if i == 0:
            validation_idx = batch
        else:
            train_idx.extend(batch)
    return train_idx, validation_idx

In [9]:
train_idx_df = pd.DataFrame()
val_idx_df = pd.DataFrame()
for i in range(5):
    train_idx, validation_idx = get_train_val()
    train_idx_df[i] = train_idx
    val_idx_df[i] = validation_idx
train_idx_df.to_csv("train_idx.csv", index=False)
val_idx_df.to_csv("val_idx.csv", index=False)
    

Load starting indices

In [10]:
train_idx_df = pd.read_csv("train_idx.csv")
val_idx_df = pd.read_csv("val_idx.csv")

## Konfiguracja sieci - VGG16

In [11]:
vgg16 = models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT) # get vgg16 model with pretrained weights

# change the number of classes in the last layer
vgg16.classifier[6].out_features = 10
# freeze convolution weights
for param in vgg16.features.parameters():
    param.requires_grad = False

input_lastLayer = vgg16.classifier[6].in_features
vgg16.classifier[6] = nn.Linear(input_lastLayer,10)
vgg16.to(device)
net = vgg16

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/mateusz/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100.0%


In [12]:
optimizer = torch.optim.Adam(net.parameters())
loss_module = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

Generate starting state

In [13]:
initial_dict = copy.deepcopy(net.state_dict())
optim_dict = copy.deepcopy(optimizer.state_dict())
sched_dict = copy.deepcopy(scheduler.state_dict())

torch.save(initial_dict, "model_dict.pt")
torch.save(optim_dict, "optimizer_dict.pt")
torch.save(sched_dict, "scheduler_dict.pt")

Load starting state

In [14]:
initial_dict =torch.load("model_dict.pt")
optim_dict = torch.load("optimizer_dict.pt")
sched_dict = torch.load("scheduler_dict.pt")

## Aktywne uczenie - pipeline

In [15]:
def validate(model, dataloader):
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    with torch.no_grad():
        for int, data in enumerate(dataloader):
            data, target = data[0].to(device), data[1].to(device)
            output = model(data)
            loss = loss_module(output, target)
            
            val_running_loss += loss.item()
            _, preds = torch.max(output.data, 1)
            val_running_correct += (preds == target).sum().item()
        val_loss = val_running_loss/len(dataloader.dataset)
        val_accuracy = 100. * val_running_correct/len(dataloader.dataset)
    
    return val_loss, val_accuracy

In [16]:
def active_learn(model, train_data, train_idx, val_idx, heuristic, initial_train_idx, experiment_id):
    model.load_state_dict(initial_dict)
    optimizer.load_state_dict(optim_dict)
    batch_len = int(0.05*len(train_idx)) # calculate length of 5% of training data
    val_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(train_data, val_idx),
        batch_size=64,
        num_workers=8)
    validation_loss, validation_acc = [], []
    train_loss, train_acc = [], []
    for epoch in range(20):
        print("Epoch: ", epoch)
    
        if epoch == 0:
        # if False:
            # batch_idx, train_idx = generate_random_sample(train_idx, batch_len) # equivalent to first (labeled) dataset
            batch_idx = initial_train_idx
            sub_epochs = 20
        else:
            for group in optimizer.param_groups:
                group['lr'] = 0.001
            old_idx = batch_idx 
            batch_idx, train_idx = generate_heuristic_sample(train_idx, batch_len, model, heuristic, train_data) # equivalent to asking for labelling
            # consider joining the samples
            batch_idx = np.append(old_idx,batch_idx)
            sub_epochs = 20
        print(f"Training on {len(batch_idx)} samples")
        
        # model.load_state_dict(initial_dict)
        # optimizer.load_state_dict(optim_dict)
        # print(model.state_dict(['conv1.weight'])[0])
        model.train()  # turn on training mode
        epoch_subset =  torch.utils.data.Subset(train_data, batch_idx) # get epoch subset 
        epoch_loader =  torch.utils.data.DataLoader(epoch_subset, batch_size=64, num_workers=8, shuffle=True) # convert into loader
        # consider retraining multiple times
        for sub_epoch in range(sub_epochs):
            total = 0   # total n of samples seen
            correct = 0   # total n of coreectly classified samples
            running_loss = 0.0
            for i, data in enumerate(epoch_loader):
                data, target = data[0].to(device), data[1].to(device)
                optimizer.zero_grad()  # zero out the gradients
                output = model(data)  # get the output of the net
                loss = loss_module(output, target)  # calculate the loss 
                running_loss += loss.item()  # add the loss on the subset batch
                _, preds = torch.max(output.data, 1)  # get the predictions
                correct += (preds == target).sum().item() # add the n of correctly classified samples
                total += target.size(0) # add the total of samples seen
                loss.backward()  # backpropagate the weights
                optimizer.step()  # optimize
                del data, target, output, preds
            t_loss = running_loss/total
            t_acc = 100. * correct/total
            val_loss, val_acc = validate(model, val_loader)
            print(f"Sub epoch {sub_epoch} train acc: {t_acc:.2f} train loss: {t_loss:.4f} val acc: {val_acc:.2f} val loss: {val_loss:.4f}")
            scheduler.step(val_loss)
            
        train_loss.append(t_loss)
        train_acc.append(t_acc)

        # val_loss, val_acc = validate(model, val_loader)
        validation_loss.append(val_loss)
        validation_acc.append(val_acc)

        print(f"Train Loss: {t_loss:.4f}, Train Acc: {t_acc:.2f} ",
          f"Validation Loss: {val_loss:.4f}, Validation Acc: {val_acc:.2f}")
        # if heuristic is not None:
        #     torch.save(model.state_dict(), f'{experiment_id}/epoch{epoch}_{heuristic.__name__}.pt')
        # else:
        #     torch.save(model.state_dict(), f'{experiment_id}/epoch{epoch}_random.pt')
    return train_loss, train_acc, validation_loss, validation_acc

In [17]:
def get_acc_per_class(model, testloader, classes):
    
    model.eval()
    # prepare to count predictions for each class
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}

    # again no gradients needed
    with torch.no_grad():
        for data in testloader:
            images, labels = data    
            images = images.to(device)
            outputs = model(images).cpu()   
            _, predictions = torch.max(outputs, 1)
            # collect the correct predictions for each class
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1


    # print accuracy for each class
    all_acc = []
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        all_acc.append(accuracy)
        print("Accuracy for class {:5s} is: {:.1f} %".format(classname, 
                                                       accuracy))
    print(f"Average accuracy is {sum(all_acc)/len(all_acc)}")

## Heurystyki

In [18]:
def largest_margin_heuristic(indices, n_samples, model, data):
    if len(indices) <= n_samples:
        return indices, []
    with torch.no_grad():
        heuristic_loader = torch.utils.data.DataLoader(
          torch.utils.data.Subset(data, indices),
          batch_size=64,
          num_workers=8
        )
        diff = np.array([])
        for data in heuristic_loader:
            data, target = data[0].to(device), data[1].to(device)
            output = model(data)
            probs = torch.softmax(output, axis=1)
            batch_diff = torch.max(probs.data, 1)[0] - torch.min(probs.data, 1)[0]
            diff = np.append(diff, batch_diff.cpu().numpy())
        #choose n_samples with smallest ?
        
        smallest = np.argpartition(diff, n_samples)[:n_samples]
        chosen = indices[smallest]
        leftover = np.setdiff1d(indices, chosen, assume_unique=True)
        return chosen, leftover

In [19]:
def smallest_margin_heuristic(indices, n_samples, model, data):
    if len(indices) <= n_samples:
        return indices, []
    with torch.no_grad():
        heuristic_loader = torch.utils.data.DataLoader(
          torch.utils.data.Subset(data, indices),
          batch_size=64,
          num_workers=8
        )
        diff = np.array([])
        for data in heuristic_loader:
            data, target = data[0].to(device), data[1].to(device)
            output = model(data)
            probs = torch.softmax(output, axis=1)
            top2 = torch.topk(probs.data, 2).values
            batch_diff = top2[:,0] - top2[:,1]
            diff = np.append(diff, batch_diff.cpu().numpy())
        #choose n_samples with smallest ?
        smallest = np.argpartition(diff, n_samples)[:n_samples]
        chosen = indices[smallest]
        leftover = np.setdiff1d(indices, chosen, assume_unique=True)
        return chosen, leftover

In [20]:
def least_confidence_heuristic(indices, n_samples, model, data):
    if len(indices) <= n_samples:
        return indices, []
    with torch.no_grad():
        heuristic_loader = torch.utils.data.DataLoader(
          torch.utils.data.Subset(data, indices),
          batch_size=64,
          num_workers=8
        )
        max_probs = np.array([])
        for data in heuristic_loader:
            data, target = data[0].to(device), data[1].to(device)
            output = model(data)
            probs = torch.softmax(output, axis=1)
            max_prob = torch.max(output.data, 1)[0]
            max_probs = np.append(max_probs, max_prob.cpu().numpy())
        #choose n_samples with smallest ?
        smallest = np.argpartition(max_probs, n_samples)[:n_samples]
        chosen = indices[smallest]
        leftover = np.setdiff1d(indices, chosen, assume_unique=True)
        return chosen, leftover

In [21]:
def mc_dropout_heuristic(indices, n_samples, model, data):
    if len(indices) <= n_samples:
        return indices, []
    with torch.no_grad():
        heuristic_loader = torch.utils.data.DataLoader(
          torch.utils.data.Subset(data, indices),
          batch_size=64,
          num_workers=8
        )
        model.eval()
        count = 0
        for m in model.modules():
            if isinstance(m, nn.Dropout) or isinstance(m, nn.Dropout2d):
                m.train(True)
                count += 1
        assert count > 0, 'We can only do models with dropout!'
        i = 0
        all_results = np.array([])
        for data in heuristic_loader:
            data, target = data[0].to(device), data[1].to(device)
            input = data.repeat(7, 1, 1, 1)
            output = model(input).data
            average_output = output.view(7, data.size(0), -1).mean(dim=0)
            probs = torch.softmax(average_output,axis=1)
            entropy = (-probs * probs.log()).sum(dim=1, keepdim=True)
            all_results = np.append(all_results, entropy.cpu().numpy())
            i+=1
        #choose n_samples with largest ?
        # print(all_results, all_results.shape)
        smallest = np.argpartition(all_results, n_samples)[-n_samples:]
        # print(smallest, smallest.shape)
        # print(smallest[:n_samples], smallest[:n_samples].shape)
        chosen = indices[smallest]
        leftover = np.setdiff1d(indices, chosen, assume_unique=True)
        return chosen, leftover

In [22]:
def score_svm_heuristic(indices, n_samples, model, data):
    if len(indices) <= n_samples:
        return indices, []
    # global svm, train_svm
    svm_results = np.array([])
    # if train_svm:
    if True:
        heuristic_loader = torch.utils.data.DataLoader(
              torch.utils.data.Subset(data, indices),
              batch_size=64,
              num_workers=8
            )
        for data in heuristic_loader:
            data, target = data[0].to(device), data[1].to(device)
            output = model(data)
            svm_result = svm(output)
            svm_results = np.append(svm_results, svm_result.detach().cpu().numpy())
        #choose n_samples with smallest ?
        smallest = np.argpartition(svm_results, n_samples)[:n_samples]
        chosen = indices[smallest]
        leftover = np.setdiff1d(indices, chosen, assume_unique=True)
        # train_svm = False
        return chosen, leftover
    else:
        with torch.no_grad():
            heuristic_loader = torch.utils.data.DataLoader(
              torch.utils.data.Subset(data, indices),
              batch_size=64,
              num_workers=8
            )
            for data in heuristic_loader:
                data, target = data[0].to(device), data[1].to(device)
                output = model(data)
                svm_result = svm(output)
                svm_results = np.append(svm_results, svm_result.cpu().numpy())
            #choose n_samples with largest ?
            smallest = np.argpartition(svm_results, n_samples)[-n_samples:]
            chosen = indices[smallest]
            leftover = np.setdiff1d(indices, chosen, assume_unique=True)
            return chosen, leftover

In [23]:
def generate_random_sample(indices, n_samples):
    chosen = np.random.choice(indices, n_samples, replace=False)
    leftover = np.setdiff1d(indices, chosen, assume_unique=True)
  # leftover = np.delete(indices, chosen)
    return chosen, leftover

In [24]:
def generate_heuristic_sample(indices, n_samples, model, heuristic=None, data=None):
    if heuristic is None:
        return generate_random_sample(indices, n_samples)
    else:
        return heuristic(indices, n_samples, model, data)

## Wyniki

## Baseline of 20 epochs on whole dataset

In [25]:
def base_learn(model, train_data, train_idx, val_idx):
    model.load_state_dict(initial_dict)
    optimizer.load_state_dict(optim_dict)
    scheduler.load_state_dict(sched_dict)
    val_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(train_data, val_idx),
        batch_size=64,
        num_workers=8)
    validation_loss, validation_acc = [], []
    train_loss, train_acc = [], []
    test_loss, test_acc = [], []
    batch_idx = train_idx
    print(f"Training on {len(batch_idx)} samples")
    sub_epochs = 20
    model.train()  # turn on training mode
    epoch_subset =  torch.utils.data.Subset(train_data, batch_idx) # get epoch subset 
    epoch_loader =  torch.utils.data.DataLoader(epoch_subset, batch_size=64, num_workers=8, shuffle=True) # convert into loader
    # consider retraining multiple times
    for sub_epoch in range(sub_epochs):
        total = 0   # total n of samples seen
        correct = 0   # total n of coreectly classified samples
        running_loss = 0.0
        for i, data in enumerate(epoch_loader):
            data, target = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()  # zero out the gradients
            output = model(data)  # get the output of the net
            loss = loss_module(output, target)  # calculate the loss 
            running_loss += loss.item()  # add the loss on the subset batch
            _, preds = torch.max(output.data, 1)  # get the predictions
            correct += (preds == target).sum().item() # add the n of correctly classified samples
            total += target.size(0) # add the total of samples seen
            loss.backward()  # backpropagate the weights
            optimizer.step()  # optimize
            del data, target, output, preds
        t_loss = running_loss/total
        t_acc = 100. * correct/total
        val_loss, val_acc = validate(model, val_loader)
        print(f"Sub epoch {sub_epoch} train acc: {t_acc:.2f} train loss: {t_loss:.4f} val acc: {val_acc:.2f} val loss: {val_loss:.4f}")
        scheduler.step(val_loss)

    train_loss.append(t_loss)
    train_acc.append(t_acc)

    validation_loss.append(val_loss)
    validation_acc.append(val_acc)
    
    ts_loss, ts_acc = validate(model, test_loader)
    test_loss.append(ts_loss)
    test_acc.append(ts_acc)

    print(f"Train Loss: {t_loss:.4f}, Train Acc: {t_acc:.2f} ",
      f"Validation Loss: {val_loss:.4f}, Validation Acc: {val_acc:.2f}")
    # if heuristic is not None:
    #     torch.save(model.state_dict(), f'{experiment_id}/epoch{epoch}_{heuristic.__name__}.pt')
    # else:
    #     torch.save(model.state_dict(), f'{experiment_id}/epoch{epoch}_random.pt')
    return train_loss, train_acc, validation_loss, validation_acc, test_loss, test_acc

In [ ]:
train_loss_df = pd.DataFrame()
train_acc_df = pd.DataFrame()
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
test_loss_df = pd.DataFrame()
test_acc_df = pd.DataFrame()
test_loader = torch.utils.data.DataLoader(
        testset,
        batch_size=64,
        num_workers=8)
for i in range(5):
    i = str(i)
    train_loss, train_acc, validation_loss, validation_acc, test_loss, test_acc = \
        base_learn(net, trainset, train_idx_df[i].to_list(), val_idx_df[i].to_list())
    train_loss_df[i] = train_loss
    train_acc_df[i] = train_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc
    test_loss_df[i] = test_loss
    test_acc_df[i] = test_acc

In [22]:
train_loss_df.to_csv("train_loss_cifar10_whole.csv")
train_acc_df.to_csv("train_acc_cifar10_whole.csv")
val_loss_df.to_csv("val_loss_cifar10_whole.csv")
val_acc_df.to_csv("val_acc_cifar10_whole.csv")
test_loss_df.to_csv("test_loss_cifar10_whole.csv")
test_acc_df.to_csv("test_acc_cifar10_whole.csv")

Generate first batch

In [56]:
batch_df = pd.DataFrame()
train_df = pd.DataFrame()
for i in range(5):
    train_idx = train_idx_df[i].to_list()
    batch_idx, train_idx = generate_random_sample(train_idx, 4*2250)
    batch_df[i] = batch_idx
    train_df[i] = train_idx
batch_df.to_csv("first_batch_idx.csv", index=False)
train_df.to_csv("other_train_idx.csv", index=False)

Load first batch

In [21]:
batch_df = pd.read_csv("first_batch_idx.csv")
train_df = pd.read_csv("other_train_idx.csv")

### Wybór losowy

In [26]:
train_loss_df = pd.DataFrame()
train_acc_df = pd.DataFrame()
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
for i in range(4, 5):
    i = str(i)
    train_loss, train_acc, validation_loss, validation_acc = \
        active_learn(net, trainset, train_df[i].to_list(), val_idx_df[i].to_list(), 
                     heuristic=None, initial_train_idx=batch_df[i].to_list(),
                     experiment_id=i)
    train_loss_df[i] = train_loss
    train_acc_df[i] = train_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc
    


Epoch:  0
Training on 9000 samples
Sub epoch 0 train acc: 57.91 train loss: 0.0210 val acc: 68.16 val loss: 0.0147
Sub epoch 1 train acc: 74.04 train loss: 0.0119 val acc: 71.66 val loss: 0.0133
Sub epoch 2 train acc: 83.28 train loss: 0.0076 val acc: 71.20 val loss: 0.0152
Sub epoch 3 train acc: 88.61 train loss: 0.0052 val acc: 71.72 val loss: 0.0159
Sub epoch 4 train acc: 92.26 train loss: 0.0037 val acc: 71.96 val loss: 0.0181
Sub epoch 5 train acc: 94.37 train loss: 0.0030 val acc: 70.68 val loss: 0.0249
Sub epoch 6 train acc: 94.21 train loss: 0.0030 val acc: 70.58 val loss: 0.0229
Sub epoch 7 train acc: 95.59 train loss: 0.0025 val acc: 71.84 val loss: 0.0266
Sub epoch 8 train acc: 95.88 train loss: 0.0022 val acc: 68.02 val loss: 0.0300
Sub epoch 9 train acc: 96.37 train loss: 0.0022 val acc: 71.50 val loss: 0.0308
Sub epoch 10 train acc: 95.77 train loss: 0.0032 val acc: 70.94 val loss: 0.0316
Sub epoch 11 train acc: 98.24 train loss: 0.0011 val acc: 73.32 val loss: 0.0255
Sub

In [27]:
train_loss_df.to_csv("t_loss_random.csv",index=False)
train_acc_df.to_csv("t_acc_random.csv",index=False)
val_loss_df.to_csv("v_loss_random.csv",index=False)
val_acc_df.to_csv("v_acc_random.csv",index=False)

In [28]:
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
test_loss_df = pd.DataFrame()
test_acc_df = pd.DataFrame()
test_loader = torch.utils.data.DataLoader(
        testset,
        batch_size=64,
        num_workers=8)
for i in range(5):
    i = str(i)
    validation_loss, validation_acc = [], []
    test_loss, test_acc = [], []
    val_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(trainset, val_idx_df[i]),
        batch_size=64,
        num_workers=8)
    for k in range(20):
        net.load_state_dict(torch.load(f"{i}/epoch{k}_random.pt"))
        
        v_loss, v_acc = validate(net, val_loader)
        t_loss, t_acc = validate(net, test_loader)
        validation_loss.append(v_loss)
        validation_acc.append(v_acc)
        test_loss.append(t_loss)
        test_acc.append(t_acc)
        print(t_acc)

    test_loss_df[i] = test_loss
    test_acc_df[i] = test_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc
    print(test_loss, test_acc)

74.42
73.85
74.32
74.23
74.41
75.05
75.63
76.29
75.93
72.1
76.02
76.25
75.82
75.96
76.35
76.2
75.22
76.3
76.73
76.38
[0.03296021600961685, 0.03289615208506584, 0.031313410532474516, 0.031788754892349246, 0.026697751504182816, 0.02954068830013275, 0.03423783921003341, 0.036069386869668964, 0.03495805795788765, 0.02910536430478096, 0.040172080296278, 0.037038950073719025, 0.046246442079544064, 0.044981201827526095, 0.046416576790809634, 0.04572185640931129, 0.052145075243711474, 0.04863161411881447, 0.04964851890802383, 0.05463846625685692] [74.42, 73.85, 74.32, 74.23, 74.41, 75.05, 75.63, 76.29, 75.93, 72.1, 76.02, 76.25, 75.82, 75.96, 76.35, 76.2, 75.22, 76.3, 76.73, 76.38]
73.59
74.5
74.19
75.29
76.0
75.94
76.05
75.5
75.26
75.53
75.86
75.92
76.28
76.12
75.14
76.08
76.51
76.06
75.86
72.63
[0.03300826088488102, 0.02850603396296501, 0.02906369135975838, 0.026388626903295517, 0.027411285662651063, 0.03235744072496891, 0.03455528582930565, 0.031149020993709566, 0.03349213059544563, 0.03468

In [29]:
val_loss_df.to_csv("val_loss_random.csv",index=False)
val_acc_df.to_csv("val_acc_random.csv",index=False)
test_loss_df.to_csv("test_loss_random.csv",index=False)
test_acc_df.to_csv("test_acc_random.csv",index=False)

### Najmniejsza pewność

In [36]:
train_loss_df = pd.DataFrame()
train_acc_df = pd.DataFrame()
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
for i in range(5):
    i = str(i)
    train_loss, train_acc, validation_loss, validation_acc = \
        active_learn(net, trainset, np.array(train_df[i].to_list()), np.array(val_idx_df[i].to_list()), 
                      heuristic=least_confidence_heuristic, initial_train_idx=np.array(batch_df[i].to_list()),
                     experiment_id=i)
    train_loss_df[i] = train_loss
    train_acc_df[i] = train_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc
    

Epoch:  0
Training on 9000 samples
Sub epoch 0 train acc: 57.07 train loss: 0.0213 val acc: 70.62 val loss: 0.0145
Sub epoch 1 train acc: 75.23 train loss: 0.0115 val acc: 71.38 val loss: 0.0132
Sub epoch 2 train acc: 83.34 train loss: 0.0074 val acc: 70.28 val loss: 0.0148
Sub epoch 3 train acc: 89.66 train loss: 0.0048 val acc: 72.10 val loss: 0.0171
Sub epoch 4 train acc: 91.91 train loss: 0.0042 val acc: 70.76 val loss: 0.0207
Sub epoch 5 train acc: 93.59 train loss: 0.0030 val acc: 70.80 val loss: 0.0229
Sub epoch 6 train acc: 95.34 train loss: 0.0024 val acc: 70.84 val loss: 0.0232
Sub epoch 7 train acc: 95.53 train loss: 0.0022 val acc: 70.48 val loss: 0.0265
Sub epoch 8 train acc: 96.17 train loss: 0.0020 val acc: 69.96 val loss: 0.0284
Sub epoch 9 train acc: 95.70 train loss: 0.0028 val acc: 69.96 val loss: 0.0276
Sub epoch 10 train acc: 96.07 train loss: 0.0025 val acc: 70.48 val loss: 0.0316
Sub epoch 11 train acc: 96.92 train loss: 0.0019 val acc: 70.62 val loss: 0.0333
Sub

KeyboardInterrupt: 

In [ ]:
train_loss_df.to_csv("t_loss_lc.csv",index=False)
train_acc_df.to_csv("t_acc_lc.csv",index=False)
val_loss_df.to_csv("v_loss_lc.csv",index=False)
val_acc_df.to_csv("v_acc_lc.csv",index=False)

In [37]:
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
test_loss_df = pd.DataFrame()
test_acc_df = pd.DataFrame()
test_loader = torch.utils.data.DataLoader(
        testset,
        batch_size=64,
        num_workers=8)
for i in range(3):
    i = str(i)
    validation_loss, validation_acc = [], []
    test_loss, test_acc = [], []
    val_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(trainset, val_idx_df[i]),
        batch_size=64,
        num_workers=8)
    for k in range(20):
        net.load_state_dict(torch.load(f"{i}/epoch{k}_least_confidence_heuristic.pt"))
        
        v_loss, v_acc = validate(net, val_loader)
        t_loss, t_acc = validate(net, test_loader)
        validation_loss.append(v_loss)
        validation_acc.append(v_acc)
        test_loss.append(t_loss)
        test_acc.append(t_acc)
        print(t_acc)

    test_loss_df[i] = test_loss
    test_acc_df[i] = test_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc
    print(test_loss, test_acc)

73.17
74.49
75.84
75.87
76.4
75.37
74.8
69.7
74.35
74.96
75.47
75.41
75.61
76.0
76.44
76.25
75.77
75.3
75.25
75.94
[0.03433993081450462, 0.04029923709630966, 0.02843662793636322, 0.032280469465255736, 0.03600971526503563, 0.028391061723232268, 0.03516269976496696, 0.03370098366737366, 0.0377767468392849, 0.043497558403015134, 0.05201376031637192, 0.04381223875880241, 0.04785399993062019, 0.05588322209715843, 0.057053257197141646, 0.062170399010181426, 0.048832720243930816, 0.0591278607070446, 0.06631128607094287, 0.06939521477818489] [73.17, 74.49, 75.84, 75.87, 76.4, 75.37, 74.8, 69.7, 74.35, 74.96, 75.47, 75.41, 75.61, 76.0, 76.44, 76.25, 75.77, 75.3, 75.25, 75.94]
73.16
75.22
76.0
76.8
76.45
75.44
73.23
74.61
75.57
75.64
75.6
75.21
75.45
75.98
76.18
76.27
75.82
75.98
75.68
76.0
[0.0350031424343586, 0.035160158413648605, 0.030203151834011076, 0.03063730726838112, 0.03678650292158127, 0.02780250666439533, 0.026958108484745027, 0.03332386157214642, 0.038417069709301, 0.0443313530266284

In [38]:
val_loss_df.to_csv("val_loss_lc.csv",index=False)
val_acc_df.to_csv("val_acc_lc.csv",index=False)
test_loss_df.to_csv("test_loss_lc.csv",index=False)
test_acc_df.to_csv("test_acc_lc.csv",index=False)

### Najmniejszy margines

In [21]:
train_loss_df = pd.DataFrame()
train_acc_df = pd.DataFrame()
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
for i in range(3):
    i = str(i)
    train_loss, train_acc, validation_loss, validation_acc = \
        active_learn(net, trainset, np.array(train_df[i].to_list()), np.array(val_idx_df[i].to_list()), 
                      heuristic=smallest_margin_heuristic, initial_train_idx=np.array(batch_df[i].to_list()),
                     experiment_id=i)
    train_loss_df[i] = train_loss
    train_acc_df[i] = train_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc

Epoch:  0
Training on 9000 samples
Sub epoch 0 train acc: 58.36 train loss: 0.0208 val acc: 70.40 val loss: 0.0143
Sub epoch 1 train acc: 74.96 train loss: 0.0116 val acc: 71.26 val loss: 0.0137
Sub epoch 2 train acc: 84.62 train loss: 0.0070 val acc: 70.60 val loss: 0.0161
Sub epoch 3 train acc: 88.66 train loss: 0.0055 val acc: 71.10 val loss: 0.0183
Sub epoch 4 train acc: 92.66 train loss: 0.0034 val acc: 70.64 val loss: 0.0219
Sub epoch 5 train acc: 93.64 train loss: 0.0032 val acc: 71.78 val loss: 0.0201
Sub epoch 6 train acc: 95.50 train loss: 0.0024 val acc: 70.86 val loss: 0.0241
Sub epoch 7 train acc: 95.04 train loss: 0.0029 val acc: 69.42 val loss: 0.0309
Sub epoch 8 train acc: 95.37 train loss: 0.0029 val acc: 70.48 val loss: 0.0298
Sub epoch 9 train acc: 95.80 train loss: 0.0025 val acc: 71.20 val loss: 0.0310
Sub epoch 10 train acc: 96.36 train loss: 0.0024 val acc: 71.68 val loss: 0.0312
Sub epoch 11 train acc: 97.50 train loss: 0.0017 val acc: 71.28 val loss: 0.0342
Sub

In [22]:
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
test_loss_df = pd.DataFrame()
test_acc_df = pd.DataFrame()
test_loader = torch.utils.data.DataLoader(
        testset,
        batch_size=64,
        num_workers=8)
for i in range(3):
    i = str(i)
    validation_loss, validation_acc = [], []
    test_loss, test_acc = [], []
    val_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(trainset, val_idx_df[i]),
        batch_size=64,
        num_workers=8)
    for k in range(20):
        net.load_state_dict(torch.load(f"{i}/epoch{k}_smallest_margin_heuristic.pt"))
        
        v_loss, v_acc = validate(net, val_loader)
        t_loss, t_acc = validate(net, test_loader)
        validation_loss.append(v_loss)
        validation_acc.append(v_acc)
        test_loss.append(t_loss)
        test_acc.append(t_acc)
        # print(t_acc)

    test_loss_df[i] = test_loss
    test_acc_df[i] = test_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc
    print(test_loss, test_acc)

[0.03600473269820213, 0.033935772067308426, 0.028197829547524454, 0.030880572390556336, 0.035645881724357606, 0.026717642000317572, 0.03283446282148361, 0.03458277314603329, 0.03717316164970398, 0.043022533565759656, 0.045501271241903306, 0.03936918632388115, 0.04831625615954399, 0.05594742188453675, 0.05798075133562088, 0.052946415758132935, 0.049080237919092176, 0.053179271894693375, 0.05563410376310349, 0.06313519349098205] [72.66, 74.58, 75.83, 76.55, 76.64, 76.77, 76.13, 75.94, 76.8, 75.47, 77.14, 75.23, 70.52, 72.39, 75.35, 76.98, 75.57, 76.07, 76.28, 76.44]
[0.03590092757940292, 0.03291630218923092, 0.028492518571019174, 0.029957186424732208, 0.03316184681653976, 0.023359164160490036, 0.029120687621831894, 0.037302550280094146, 0.04570304360985756, 0.04740715538859367, 0.053437594997882844, 0.04205821812748909, 0.04642532378435135, 0.04677976279258728, 0.05638925438523293, 0.055658780252933505, 0.045087338984012605, 0.05926678167581558, 0.057923810636997225, 0.059769255661964414

In [23]:
val_loss_df.to_csv("val_loss_sm.csv",index=False)
val_acc_df.to_csv("val_acc_sm.csv",index=False)
test_loss_df.to_csv("test_loss_sm.csv",index=False)
test_acc_df.to_csv("test_acc_sm.csv",index=False)

### Największy margines

In [24]:
train_loss_df = pd.DataFrame()
train_acc_df = pd.DataFrame()
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
for i in range(3):
    i = str(i)
    train_loss, train_acc, validation_loss, validation_acc = \
        active_learn(net, trainset, np.array(train_df[i].to_list()), np.array(val_idx_df[i].to_list()), 
                      heuristic=largest_margin_heuristic, initial_train_idx=np.array(batch_df[i].to_list()),
                     experiment_id=i)
    train_loss_df[i] = train_loss
    train_acc_df[i] = train_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc

Epoch:  0
Training on 9000 samples
Sub epoch 0 train acc: 58.39 train loss: 0.0208 val acc: 69.32 val loss: 0.0141
Sub epoch 1 train acc: 75.23 train loss: 0.0114 val acc: 71.04 val loss: 0.0141
Sub epoch 2 train acc: 84.29 train loss: 0.0072 val acc: 70.86 val loss: 0.0159
Sub epoch 3 train acc: 89.89 train loss: 0.0047 val acc: 71.66 val loss: 0.0176
Sub epoch 4 train acc: 92.91 train loss: 0.0035 val acc: 69.86 val loss: 0.0231
Sub epoch 5 train acc: 97.62 train loss: 0.0012 val acc: 74.28 val loss: 0.0195
Sub epoch 6 train acc: 99.81 train loss: 0.0003 val acc: 74.48 val loss: 0.0216
Sub epoch 7 train acc: 99.97 train loss: 0.0001 val acc: 74.42 val loss: 0.0241
Sub epoch 8 train acc: 99.99 train loss: 0.0000 val acc: 74.48 val loss: 0.0263
Sub epoch 9 train acc: 100.00 train loss: 0.0000 val acc: 74.22 val loss: 0.0282
Sub epoch 10 train acc: 100.00 train loss: 0.0000 val acc: 74.46 val loss: 0.0296
Sub epoch 11 train acc: 100.00 train loss: 0.0000 val acc: 74.48 val loss: 0.0309


In [25]:
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
test_loss_df = pd.DataFrame()
test_acc_df = pd.DataFrame()
test_loader = torch.utils.data.DataLoader(
        testset,
        batch_size=64,
        num_workers=8)
for i in range(3):
    i = str(i)
    validation_loss, validation_acc = [], []
    test_loss, test_acc = [], []
    val_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(trainset, val_idx_df[i]),
        batch_size=64,
        num_workers=8)
    for k in range(20):
        net.load_state_dict(torch.load(f"{i}/epoch{k}_largest_margin_heuristic.pt"))
        
        v_loss, v_acc = validate(net, val_loader)
        t_loss, t_acc = validate(net, test_loader)
        validation_loss.append(v_loss)
        validation_acc.append(v_acc)
        test_loss.append(t_loss)
        test_acc.append(t_acc)
        # print(t_acc)

    test_loss_df[i] = test_loss
    test_acc_df[i] = test_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc
    print(test_loss, test_acc)

[0.034221128416061404, 0.030021648013591768, 0.03352630383372307, 0.03390977293252945, 0.026681521824002265, 0.028101584720611573, 0.03072965383529663, 0.03829390384554863, 0.043164082270860674, 0.03102817734479904, 0.037251583501696585, 0.045133573585748675, 0.046611093801259995, 0.05763256804943085, 0.05252140027284622, 0.04664248335361481, 0.0533883596599102, 0.05617020137310028, 0.06763856542110443, 0.07213651555776596] [73.75, 75.03, 75.47, 76.04, 76.59, 76.97, 76.9, 76.84, 77.24, 75.29, 76.81, 76.33, 76.49, 77.19, 77.09, 76.69, 76.91, 76.56, 76.78, 77.23]
[0.024642067444324494, 0.026294793757796288, 0.029149493637681007, 0.031235499972105025, 0.033427664077281954, 0.02712154423892498, 0.0284332361638546, 0.03337884835600853, 0.0381227966606617, 0.04165648850202561, 0.035259668678045274, 0.03836000688672066, 0.03909743628501892, 0.046916135156154636, 0.04686049531698227, 0.05172883361577987, 0.0413980061352253, 0.0555210991024971, 0.05711954696178436, 0.06398370572924614] [75.13, 

In [26]:
val_loss_df.to_csv("val_loss_lm.csv",index=False)
val_acc_df.to_csv("val_acc_lm.csv",index=False)
test_loss_df.to_csv("test_loss_lm.csv",index=False)
test_acc_df.to_csv("test_acc_lm.csv",index=False)

### Monte Carlo

In [22]:
train_loss_df = pd.DataFrame()
train_acc_df = pd.DataFrame()
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
for i in range(3):
    i = str(i)
    train_loss, train_acc, validation_loss, validation_acc = \
        active_learn(net, trainset, np.array(train_df[i].to_list()), np.array(val_idx_df[i].to_list()), 
                      heuristic=mc_dropout_heuristic, initial_train_idx=np.array(batch_df[i].to_list()),
                     experiment_id=i)
    train_loss_df[i] = train_loss
    train_acc_df[i] = train_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc

Epoch:  0
Training on 9000 samples
Sub epoch 0 train acc: 58.38 train loss: 0.0207 val acc: 66.12 val loss: 0.0160
Sub epoch 1 train acc: 74.99 train loss: 0.0117 val acc: 71.42 val loss: 0.0136
Sub epoch 2 train acc: 84.07 train loss: 0.0071 val acc: 72.28 val loss: 0.0141
Sub epoch 3 train acc: 90.11 train loss: 0.0047 val acc: 70.64 val loss: 0.0175
Sub epoch 4 train acc: 92.48 train loss: 0.0036 val acc: 70.42 val loss: 0.0218
Sub epoch 5 train acc: 93.91 train loss: 0.0031 val acc: 71.38 val loss: 0.0218
Sub epoch 6 train acc: 94.67 train loss: 0.0029 val acc: 71.86 val loss: 0.0236
Sub epoch 7 train acc: 95.24 train loss: 0.0029 val acc: 70.24 val loss: 0.0271
Sub epoch 8 train acc: 96.92 train loss: 0.0019 val acc: 72.58 val loss: 0.0273
Sub epoch 9 train acc: 97.77 train loss: 0.0013 val acc: 71.20 val loss: 0.0346
Sub epoch 10 train acc: 96.67 train loss: 0.0021 val acc: 70.62 val loss: 0.0330
Sub epoch 11 train acc: 96.08 train loss: 0.0025 val acc: 69.64 val loss: 0.0374
Sub

In [23]:
val_loss_df = pd.DataFrame()
val_acc_df = pd.DataFrame()
test_loss_df = pd.DataFrame()
test_acc_df = pd.DataFrame()
test_loader = torch.utils.data.DataLoader(
        testset,
        batch_size=64,
        num_workers=8)
for i in range(3):
    i = str(i)
    validation_loss, validation_acc = [], []
    test_loss, test_acc = [], []
    val_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(trainset, val_idx_df[i]),
        batch_size=64,
        num_workers=8)
    for k in range(20):
        net.load_state_dict(torch.load(f"{i}/epoch{k}_mc_dropout_heuristic.pt"))
        
        v_loss, v_acc = validate(net, val_loader)
        t_loss, t_acc = validate(net, test_loader)
        validation_loss.append(v_loss)
        validation_acc.append(v_acc)
        test_loss.append(t_loss)
        test_acc.append(t_acc)
        # print(t_acc)

    test_loss_df[i] = test_loss
    test_acc_df[i] = test_acc
    val_loss_df[i] = validation_loss
    val_acc_df[i] = validation_acc
    print(test_loss, test_acc)

[0.03476901218891144, 0.027744271695613863, 0.027393512743711473, 0.029083820998668672, 0.03270786690711975, 0.024515389153361322, 0.029041689616441727, 0.0334512737095356, 0.03872761110067367, 0.047458531880378726, 0.04890302317738533, 0.04128236263394356, 0.04413198601603508, 0.05052646391987801, 0.055763659423589705, 0.05958189576864242, 0.041430892074108126, 0.04460575308203697, 0.05248109845519066, 0.057188805943727494] [73.08, 74.74, 74.86, 75.52, 75.35, 74.63, 73.69, 74.11, 73.68, 73.02, 74.85, 75.19, 75.06, 75.53, 75.22, 76.13, 75.5, 75.3, 75.01, 75.87]
[0.032760739541053775, 0.03404114573001862, 0.0265092332392931, 0.031002655723690986, 0.03382639134377241, 0.028394280207157136, 0.03049158247113228, 0.03542353927493096, 0.03977607781291008, 0.046010958194732665, 0.046014554327726366, 0.04045407992601395, 0.04587230999469757, 0.04625712803602219, 0.04844567204713821, 0.05371691730618477, 0.04148171659708023, 0.04878051866292953, 0.05036159048676491, 0.049109748327732086] [73.02

In [24]:
val_loss_df.to_csv("val_loss_mc.csv",index=False)
val_acc_df.to_csv("val_acc_mc.csv",index=False)
test_loss_df.to_csv("test_loss_mc.csv",index=False)
test_acc_df.to_csv("test_acc_mc.csv",index=False)

### Maszyna wektorów nośnych

In [ ]:
# train_loss_df = pd.DataFrame()
# train_acc_df = pd.DataFrame()
# val_loss_df = pd.DataFrame()
# val_acc_df = pd.DataFrame()
# for i in range(3):
#     i = str(i)
#     train_loss, train_acc, validation_loss, validation_acc = \
#         active_learn(net, trainset, np.array(train_df[i].to_list()), np.array(val_idx_df[i].to_list()), 
#                       heuristic=mc_dropout_heuristic, initial_train_idx=np.array(batch_df[i].to_list()),
#                      experiment_id=i)
#     train_loss_df[i] = train_loss
#     train_acc_df[i] = train_acc
#     val_loss_df[i] = validation_loss
#     val_acc_df[i] = validation_acc

In [ ]:
# val_loss_df = pd.DataFrame()
# val_acc_df = pd.DataFrame()
# test_loss_df = pd.DataFrame()
# test_acc_df = pd.DataFrame()
# test_loader = torch.utils.data.DataLoader(
#         testset,
#         batch_size=64,
#         num_workers=8)
# for i in range(3):
#     i = str(i)
#     validation_loss, validation_acc = [], []
#     test_loss, test_acc = [], []
#     val_loader = torch.utils.data.DataLoader(
#         torch.utils.data.Subset(trainset, val_idx_df[i]),
#         batch_size=64,
#         num_workers=8)
#     for k in range(20):
#         net.load_state_dict(torch.load(f"{i}/epoch{k}_mc_dropout_heuristic.pt"))
        
#         v_loss, v_acc = validate(net, val_loader)
#         t_loss, t_acc = validate(net, test_loader)
#         validation_loss.append(v_loss)
#         validation_acc.append(v_acc)
#         test_loss.append(t_loss)
#         test_acc.append(t_acc)
#         # print(t_acc)

#     test_loss_df[i] = test_loss
#     test_acc_df[i] = test_acc
#     val_loss_df[i] = validation_loss
#     val_acc_df[i] = validation_acc
#     print(test_loss, test_acc)

In [ ]:
# val_loss_df.to_csv("val_loss_mc.csv",index=False)
# val_acc_df.to_csv("val_acc_mc.csv",index=False)
# test_loss_df.to_csv("test_loss_mc.csv",index=False)
# test_acc_df.to_csv("test_acc_mc.csv",index=False)